# Data Wrangling

This notebook presents different data wrangling techniques used commonly

In [1]:
# import required libraries
import random
import datetime 
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

pd.options.mode.chained_assignment = None

## Utilities

In [2]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """
    
    # sentinels
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    
    # base list of attributes
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    # introduce missing values
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        
    
    # create data frame
    df = pd.DataFrame(data_dict)
    
    return df
    

def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

## Generate a Sample Dataset

In [3]:
#sample dataset
df = generate_sample_data(row_count=1000)

### Describe the Dataset

In [5]:
#describe dataset
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int32
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int32
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 59


Sample Indices with missing data:: [0, 2, 3, 7, 9]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 969 non-null float64


C:\Users\thispc\Anaconda3\Anaconda\lib\site-packages\ipykernel_launcher.py:98: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
C:\Users\thispc\Anaconda3\Anaconda\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,20,NaN,n
1,1001,2016-01-29,5981,157,9,1226.32,n
2,1002,NaN,5650,963,15,442.88,n
3,1003,NaN,5609,512,20,5026.56,n
4,1004,2016-09-02,5010,665,6,1321.28,n


### Rename Columns

In [6]:
#display columns
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [7]:
#clean up columns
cleanup_column_names(df)

In [8]:
#dsiplay dataframe columns
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


### Sort Rows on defined attributes

In [9]:
#sort rows
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
486,-1,2016-11-02,5411,803,38,5569.09,c
115,-1,2016-06-02,5238,366,5,5325.05,b
587,-1,2016-01-26,5224,396,37,5285.69,a
586,-1,2016-01-26,5411,803,33,4402.26,d
917,-1,2016-04-01,5289,272,8,4319.79,d


### Rearrange Columns in a Dataframe

In [10]:
#rearrange columns
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,20,NaN
1,1001,2016-01-29,5981,n,157,9,1226.32
2,1002,NaN,5650,n,963,15,442.88
3,1003,NaN,5609,n,512,20,5026.56
4,1004,2016-09-02,5010,n,665,6,1321.28


### Filtering Columns

Using Column Index

In [11]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[   0  157  963  512  665  258  364 1001  766  527]


Using Column Name

In [12]:
# print 10 values of quantity purchased
print(df.quantity_purchased.values[0:10])

[20  9 15 20  6 38 16 21 13  8]


Using Column Datatype

In [13]:
# print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])

[    nan 1226.32  442.88 5026.56 1321.28 4812.39  540.04 6407.54  114.53
  621.54]


### Filtering Rows

Select specific rows

In [15]:
#select specific rows
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-01-20,5670,222,27,439.89,n
501,1501,2016-01-26,5981,157,14,2433.32,b
20,1020,NaN,5484,941,26,NaN,n


Exclude Specific Row indices

In [16]:
#drop specific rows
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,2016-01-29,5981,157,9,1226.32,n
2,1002,NaN,5650,963,15,442.88,n
3,1003,NaN,5609,512,20,5026.56,n
4,1004,2016-09-02,5010,665,6,1321.28,n
5,1005,2016-01-24,5072,258,38,4812.39,n


Conditional Filtering

In [17]:
#conditional filtering of quantity purchased > 25
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
5,1005,2016-01-24,5072,258,38,4812.39,n
10,1010,2016-01-20,5670,222,27,439.89,n
14,1014,2016-12-01,5686,392,30,1580.79,n
16,1016,2016-08-02,5626,299,29,5006.60,n
17,1017,2016-01-27,5289,272,33,3949.12,n


Offset from top of the dataframe

In [19]:
#display 5 rows starting from 100th row.
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-01-20,5480,659,7,1804.80,c
101,1101,2016-01-23,5981,157,22,1007.73,a
102,1102,2016-01-01,5650,963,9,2075.58,c
103,1103,2016-01-30,5609,512,19,26.06,d
104,1104,2016-01-29,5010,665,9,4060.35,a


Offset from bottom of the dataframe

In [20]:
#last rows
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-01-20,5164,597,30,4053.35,a
991,1991,2016-04-01,5690,147,35,3847.17,b
992,1992,2016-04-01,5897,424,20,4121.24,d
993,1993,2016-10-02,5890,284,36,2321.53,d
994,1994,2016-05-01,5331,326,3,682.53,a


### TypeCasting/Data Type Conversion

In [21]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
print(df.dtypes)

serial_no                      int32
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int32
price                        float64
user_type                     object
dtype: object


### Apply/Map Usage

Map : Create a derived attribute using map

In [22]:
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-10-01,5435,267,2,3661.90,b,new
996,1996,2016-01-02,5471,409,28,396.96,b,new
997,1997,2016-02-02,5063,922,12,3524.29,c,existing
998,1998,2016-11-02,5529,657,7,4091.14,d,loyal_existing
999,1999,2016-01-25,5312,248,17,2342.45,d,loyal_existing


Apply: Using apply to get attribute ranges

In [23]:
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

serial_no             2000.00
user_id               6093.00
product_id            1065.00
quantity_purchased      40.00
price                 8654.76
dtype: float64

Applymap: Extract week from date

In [24]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

In [26]:
#display the top rows
display(df.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,20,NaN,n,error,0
1,1001,2016-01-29,5981,157,9,1226.32,n,error,4
2,1002,NaT,5650,963,15,442.88,n,error,0
3,1003,NaT,5609,512,20,5026.56,n,error,0
4,1004,2016-09-02,5010,665,6,1321.28,n,error,35


### Missing Values

Drop Rows with missing dates

In [27]:
#drop the rows with missing dates
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
1,1001,2016-01-29,5981,157,9,1226.32,n,error,4
4,1004,2016-09-02,5010,665,6,1321.28,n,error,35
5,1005,2016-01-24,5072,258,38,4812.39,n,error,3
6,1006,2016-01-24,5495,364,16,540.04,n,error,3
8,1008,2016-08-02,5471,766,13,114.53,n,error,31


Fill Missing Price values with mean price

In [28]:
#fill the dropped values with mean values
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

Fill Missing user_type values with value from previous row (forward fill) 

In [29]:
#fill the missing values with the previous row values
df_dropped['user_type'].fillna(method='ffill',inplace=True)

Fill Missing user_type values with value from next row (backward fill)

In [30]:
#fiill the missing values with next row values
df_dropped['user_type'].fillna(method='bfill',inplace=True)

### Duplicates

Drop Duplicate serial_no rows

In [31]:
# sample duplicates
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
98,-1,2016-04-01,5529,657,16,3441.70,c,existing,13
115,-1,2016-06-02,5238,366,5,5325.05,b,new,22
120,-1,2016-04-01,5484,941,6,3509.88,b,new,13
128,-1,2016-07-02,5676,149,38,797.66,a,new,26
178,-1,2016-10-01,5663,663,20,2707.60,a,new,39


Shape of df=(969, 9)


In [33]:
#drop the duplicates
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [34]:
# updated dataframe
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
1,1001,2016-01-29,5981,157,9,1226.32,n,error,4
4,1004,2016-09-02,5010,665,6,1321.28,n,error,35
5,1005,2016-01-24,5072,258,38,4812.39,n,error,3
6,1006,2016-01-24,5495,364,16,540.04,n,error,3
8,1008,2016-08-02,5471,766,13,114.53,n,error,31


Shape of df=(940, 9)


Remove rows which have less than 3 attributes with non-missing data

In [36]:
#Remove rows which have less than 3 attributes with non-missing data
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,20,NaN,n,error,0
1,1001,2016-01-29,5981,157,9,1226.32,n,error,4
2,1002,NaT,5650,963,15,442.88,n,error,0
3,1003,NaT,5609,512,20,5026.56,n,error,0
4,1004,2016-09-02,5010,665,6,1321.28,n,error,35


Shape of df=(1000, 9)


### Encode Categoricals

One Hot Encoding using get_dummies()

In [38]:
#one hot encoding
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,NaT,-101,0,20,NaN,error,0,0,0,0,0,1
1,1001,2016-01-29,5981,157,9,1226.32,error,4,0,0,0,0,1
2,1002,NaT,5650,963,15,442.88,error,0,0,0,0,0,1
3,1003,NaT,5609,512,20,5026.56,error,0,0,0,0,0,1
4,1004,2016-09-02,5010,665,6,1321.28,error,35,0,0,0,0,1


Label Mapping

In [39]:
#label mapping
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
995,1995,2016-10-01,5435,267,2,3661.90,b,new,39,1.0
996,1996,2016-01-02,5471,409,28,396.96,b,new,53,1.0
997,1997,2016-02-02,5063,922,12,3524.29,c,existing,5,2.0
998,1998,2016-11-02,5529,657,7,4091.14,d,loyal_existing,44,3.0
999,1999,2016-01-25,5312,248,17,2342.45,d,loyal_existing,4,3.0


### Random Sampling data from DataFrame

In [40]:
#random sampling data
display(df.sample(frac=0.2, replace=True, random_state=42).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
102,1102,2016-01-01,5650,963,9,2075.58,c,existing,53,2.0
435,1435,2016-04-01,5155,894,14,4376.77,d,loyal_existing,13,3.0
860,1860,2016-01-28,5692,289,10,1364.45,b,new,4,1.0
270,-1,2016-03-02,5284,476,26,888.74,b,new,9,1.0
106,1106,2016-05-01,5495,364,18,489.67,a,new,17,0.0


### Normalizing Numeric Values

Normalize price values using  **Min-Max Scaler**

In [41]:
#min-max scaler
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [42]:
#display the nrmalized top rows
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
28,1028,2016-06-02,5676,149,1,0.652789,b,new,22,1.0
30,1030,2016-01-18,5945,1027,22,0.352760,c,existing,3,2.0
32,1032,2016-01-16,5426,220,9,0.353634,c,existing,2,2.0
35,1035,2016-02-02,5155,894,6,0.178004,b,new,5,1.0
38,1038,2016-05-01,5441,878,17,0.029111,b,new,17,1.0


Normalize quantity purchased values using  **Robust Scaler**

In [43]:
#robust scaler
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler.fit_transform(df_normalized['quantity_purchased'].values.reshape(-1,1))
df_normalized['quantity_purchased'] = rs_scaled.reshape(-1,1)

In [44]:
#display the top normalized rows
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
28,1028,2016-06-02,5676,149,-0.909091,5651.26,b,new,22,1.0
30,1030,2016-01-18,5945,1027,0.045455,3054.58,c,existing,3,2.0
32,1032,2016-01-16,5426,220,-0.545455,3062.15,c,existing,2,2.0
35,1035,2016-02-02,5155,894,-0.681818,1542.11,b,new,5,1.0
38,1038,2016-05-01,5441,878,-0.181818,253.48,b,new,17,1.0


### Data Summarization

Condition based aggregation

In [45]:
#display
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2327.0167841409693


Condtion based counts

In [46]:
#display purchase week column with the respective counts
print(df['purchase_week'].value_counts())

4     167
3     147
2     123
39     57
31     56
13     54
53     45
35     45
26     42
22     42
44     40
9      39
5      39
0      31
17     28
18     23
48     22
Name: purchase_week, dtype: int64


### Group By

Group By certain attributes

In [47]:
#group the data by sum
print(df.groupby(['user_class'])['quantity_purchased'].sum())

user_class
error              625
existing          5639
loyal_existing    4925
new               9806
Name: quantity_purchased, dtype: int32


Group By with different aggregate functions

In [48]:
#group by sum, mean an count_nonzero
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,625,20.833333,30
existing,5639,20.962825,269
loyal_existing,4925,20.780591,237
new,9806,21.133621,464


Group by specific aggregate functions for each attribute

In [49]:
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2045.648929                  41
existing       c          2486.412386                  41
loyal_existing d          2337.354844                  41
new            a          2327.016784                  41
               b          2376.253822                  41

Group by with multiple agg for each attribute

In [50]:
display(df.groupby(['user_class','user_type']).agg({'price':{
                                                                'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum}))  

C:\Users\thispc\Anaconda3\Anaconda\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            57278.17  2045.648929    1822.633967   30.0   
existing       c           656412.87  2486.412386    1648.043307  269.0   
loyal_existing d           525904.84  2337.354844    1553.062009  237.0   
new            a           528232.81  2327.016784    1597.516298  232.0   
               b           534657.11  2376.253822    1604.347309  232.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        625  
existing       c                       5639  
loyal_existing d                       4925  
new            a                       4822  
               b                       4984

### Pivot Tables

In [51]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,2013.540000,2515.810000,2250.796250,1747.470000,3886.570
2016-01-02,3575.453333,1900.226000,1511.026667,925.233333,1857.750
2016-01-13,1755.513333,2749.245000,2053.864286,2427.987143,NaN
2016-01-14,2651.861667,3226.407143,1617.847143,1858.023333,NaN
2016-01-15,2694.134000,1379.431429,3312.136000,2197.886000,1168.480
2016-01-16,1681.288333,2763.190000,2534.184286,2916.380000,NaN
2016-01-17,3609.922000,1483.776000,2687.351111,1463.140000,NaN
2016-01-18,1599.661667,3016.678333,3371.907500,1798.165714,NaN
2016-01-19,1571.970000,1998.764286,3226.735000,3207.792500,NaN


### Stack a Dataframe

In [52]:
print(df.stack())

0    serial_no                            1000
     user_id                              -101
     product_id                              0
     quantity_purchased                     20
     user_type                               n
     user_class                          error
     purchase_week                           0
1    serial_no                            1001
     date                  2016-01-29 00:00:00
     user_id                              5981
     product_id                            157
     quantity_purchased                      9
     price                             1226.32
     user_type                               n
     user_class                          error
     purchase_week                           4
2    serial_no                            1002
     user_id                              5650
     product_id                            963
     quantity_purchased                     15
     price                              442.88
     user_typ